# Design Study C: Satellite

In [1]:
import sympy
from sympy import *
from sympy.physics.vector.printing import vlatex
from IPython.display import Math, display

init_printing()

def dotprint(expr):
    display(Math(vlatex(expr)))

### Define Symbols and Configuration Variables

In [2]:
# Time symbol
t = symbols('t')

In [3]:
# Generalized coordinates

theta, phi = symbols(r'theta, phi', cls=Function)
theta = theta(t)
phi = phi(t)

In [4]:
# Derivatives of Generalized Coordinates

theta_dot = theta.diff(t)
phi_dot = phi.diff(t)

In [5]:
# Other symbols (mass, forces, damping coefficients, torques...)

Js, Jp, k, tau_theta, b  = symbols(r'J_s, J_p, k, tau_\theta, b', real=True)

### Kinetic Energy

#### Translational Kinetic Energy

In [13]:
K_T = 1/2*Js*theta_dot**2

dotprint(K_T)

<IPython.core.display.Math object>

#### Rotational Kinetic Energy

In [14]:
K_R = 1/2*Jp*theta_dot**2

dotprint(K_R)

<IPython.core.display.Math object>

#### Total Kinetic Energy

In [15]:
K = K_T + K_R

dotprint(K)

<IPython.core.display.Math object>

### Potential Energy

In [16]:
P = 1/2*K*(phi-theta)**2

dotprint(P)

<IPython.core.display.Math object>

### Define the Lagrangian

For this class, the Lagrangian will in general be a scalar value equal to the difference between kinetic and potential energy.

In [17]:
L = K - P

dotprint(L)

<IPython.core.display.Math object>

### Define the Forces

#### Generalized Forces

The forces will form a vector with the same length as the number of configuration variables. 

The first entry corresponds to the first configuration variable, the second entry to the second, etc.

Each entry should contain the generalized forces acting on the corresponding configuration variable.

In [8]:
tau = Matrix([tau_theta, theta])

dotprint(tau)

<IPython.core.display.Math object>

#### Non-conservative Forces (e.g. Damping)

Similar to the generalized forces, this will be a vector with the same length as the number of configuration variables.

Each entry should contain the nonconservative forces acting on the corresponding configuration variable.

In [9]:
B = Matrix([[b , -b], [-b, b]])
q_dot = Matrix([theta_dot, phi_dot])

Bqdot = B @ q_dot

dotprint(Bqdot)

<IPython.core.display.Math object>

In [10]:
F_total = tau + Bqdot

dotprint(F_total)

<IPython.core.display.Math object>

### Compute the Lagrangian Derivatives / Partial Derivatives

The following function computes these two terms:

* $\frac{d}{dt} \left(\frac{\partial L(q, \dot{q})}{\partial \dot{q}} \right)$
* $\frac{\partial L(q,\dot{q})}{\partial q}$

Things to note:

* `q` and `q_dot` should be Python tuples containing your configuration variables (e.g. `(x, theta)`)
* The function returns a Python list. You probably want to wrap the output in a Sympy `Matrix()`. For example,

```python
out = derive_Lagrangian(...)
out = Matrix(out)
```

In [11]:
def derive_Lagrangian(L, q, q_dot):
    term_1 = (sympy.tensor.derive_by_array(L, q_dot)).diff(t)
    term_2 = sympy.tensor.derive_by_array(L, q)
    return term_1 - term_2

### Get the final Euler Lagrange Equations of Motion

Remember that the Euler Lagrange Equations used in class are:

$$
\underbrace{\frac{d}{dt} \left(\frac{\partial L(q, \dot{q})}{\partial \dot{q}} \right) - \frac{\partial L(q,\dot{q})}{\partial q}}_\text{Left Hand Side (LHS)} = \underbrace{\tau - B \dot{q}}_\text{Right Hand Side (RHS)}.
$$

In Sympy, we use the `Eq(LHS, RHS)` class to get a symbolic equation $\text{LHS} = \text{RHS}$.

Both the left and right expressions should be Sympy vectors (i.e. Matrices with 1 column). 

The length of the left and right vectors must both be equal to the number of configuration variables.

In [18]:
LHS = derive_Lagrangian(L, [theta, phi], [theta_dot, phi_dot])
RHS = F_total
Euler_Lagrange = Eq(LHS, RHS)

dotprint(simplify(Euler_Lagrange))

<IPython.core.display.Math object>